In [5]:
import pickle 
import csv
import numpy as np

with open (f"/home/tsaibw/Multi_scale/pre_process/hand_crafted_v3.csv", mode='r', encoding='utf-8') as f:
    file = csv.reader(f, delimiter=',')
    hand_craft = list(file)
    
with open(f"/home/tsaibw/Multi_scale/pre_process/allreadability.pickle", "rb") as f:
    readability = pickle.load(f)
    

In [6]:
print(readability[0])

[1.         0.47361136 0.51242417 0.41745715 0.56747815 0.39605389
 0.51401774 0.54984141 0.40764331 0.24500106 0.40405317 0.38893406
 0.44884299 0.24285714 0.32036852 0.39883753 0.374211   0.42465753
 0.46045198 0.24285714 0.         0.31052632 0.1969697  0.29918033
 0.20408163 0.14285714 0.31818182 0.39130435 0.47008547 0.07317073
 0.08       0.28571429 0.         0.33333333 0.         0.        ]


In [7]:
chunk_sizes = [90,30,130,10]
def process_bert_inputs(inputs):
    valid_lengths = []

    for batch in inputs:
        valid_length = 0

        for idxx, element in enumerate(batch):
            input_id, token_type, mask = element

            # 檢查input_id是否為全0
            if not torch.all(input_id == 0):
                valid_length += 1
            else:
                input_id[0] = 101  # CLS token
                input_id[1] = 102  # SEP token
                token_type[:] = 0
                mask[:2] = 1
                batch[idxx] = torch.stack((input_id, token_type, mask))

        valid_lengths.append(valid_length)

    return inputs, valid_lengths


def adds(data, mode, epoch):
    attribute = ['score',
            'content',
            'organization',
            'word_choice',
            'sentence_fluency',
            'conventions',
            'prompt_adherence',
            'language',
            'narrativity']
    essay_id, prompt_id, score, text, craft, read, inputs_chunked = [], [], [], [], [], [], []
    
    for i in data:
        essay_id.append(int(i['essay_id']))
        prompt_id.append(int(i['prompt_id']))
        text.append(i['content_text'])
        att = [-1]*9
        for j in i.keys():
            if j in attribute:
                idx = attribute.index(j)
                att[idx] = int(i[j])
        score.append(att)
        
        id_ = i['essay_id']
        for j in hand_craft:
            if j[0] == id_:
                craft.append(j[2:])
                break
    
        for j in readability:
            if int(j[0]) == int(id_):
                read.append(list(j[1:]))
                break

    labels = list(score)
    test_documents = list(text)
    ids = list(prompt_id)
    # normalized
    scale_score = get_scaled_down_scores(labels, ids)
    
    inputs_single, _ = encode_documents(test_documents, tokenizer, max_input_length=512)
    # segment_encode
    for chunk_size in chunk_sizes:
        input_enc_chunk, _ = encode_documents(test_documents, tokenizer, max_input_length=chunk_size)
        inputs_chunked.append(input_enc_chunk)
    
    
    # 處理輸入
    processed_inputs, valid_lengths = [], []
    for batch in inputs_chunked:
        processed_batch, lengths = process_bert_inputs(batch)
        processed_inputs.append(processed_batch)
        valid_lengths.append(lengths)

    # 輸出處理結果
    print(f"Valid lengths : {valid_lengths}")
    out = {
        'inputs_single': inputs_single,
        'inputs_chunked': processed_inputs,
        'scaled_score': scale_score,
        'prompt_id': ids,
        'valid_length': valid_lengths,
        'hand_craft': craft,
        'readability': read,
        'essay_id': essay_id
    }
    # data2 = (inputs_single, processed_inputs, labels, ids, valid_lengths)
    with open(f"/home/tsaibw/Multi_scale/dataset/new_{mode}/encode_prompt_{epoch}.pkl", 'wb') as f:
        pickle.dump(out, f)
    
    return 

In [28]:
import pickle
with open(f"/home/tsaibw/Multi_scale/dataset/new_train/encode_prompt_1.pkl", "rb") as f:
        data = pickle.load(f)

In [32]:
# for i in data.keys():
#     print(i, data[i][2])

In [11]:
# data set
import sys
import os
import pickle
import random
import torch
from transformers import BertModel, BertConfig, BertTokenizer
from scale import  get_scaled_down_scores
from tqdm import tqdm 
sys.path.append(os.path.abspath(".."))
from data.encode import encode_documents

random.seed(0)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

for i in tqdm(range(1, 9)):
    with open(f"/home/tsaibw/Multi_scale/pre_process/cross_prompt_attributes/{i}/train.pk", "rb") as f:
        data = pickle.load(f)
    with open(f"/home/tsaibw/Multi_scale/pre_process/cross_prompt_attributes/{i}/dev.pk", "rb") as f:
        dev_data = pickle.load(f)
    with open(f"/home/tsaibw/Multi_scale/pre_process/cross_prompt_attributes/{i}/test.pk", "rb") as f:
        test_data = pickle.load(f)
    
    adds(data, 'train', i)
    adds(dev_data, 'dev', i)
    adds(test_data, 'test', i)



In [ ]:
import pickle
%cd '/content/drive/MyDrive/hw_ner'
max_len = 1024
for j in range(1, 9):
  ids = []
  texts = []
  labels = []
  for i in range (1, 9):
    if j == i:
      continue
    with open(f"/content/drive/MyDrive/hw_ner/ASAP/dataset/Train_prompt/train_prompt_{i}.txt") as file:
      for lines in file:
        line = lines.strip()
        line_vec = line.split("\t")
        if len(line_vec) == 3:
            ids.append(i)
            if len(line_vec[1].split(" ")) >= max_len:
                line_vec[1] = " ".join(line_vec[1].split(" ")[0:max_len])
            texts.append(line_vec[1])
            labels.append(float(line_vec[2]))
  data_to_save = (ids, texts, labels)
  with open(f"/multi_scale/train/cross_prompt_p{j}.pkl", 'wb') as f:
      pickle.dump(data_to_save, f)

